In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.fft
import csv


In [5]:
#Load data
path = 'https://github.com/larsRikard/Machine-learning-and-sensor-technology/blob/main/Assignment%201/Railroad%20predictive%20maintenance.ipynb'
with open(path, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    headers = next(reader)
    data = np.array(list(reader)).astype(float)

ValueError: could not convert string to float: '{'